<a href="https://colab.research.google.com/github/JdoubleU2/CareConnect/blob/Training_development/finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Dependencies & packages 
#ALL OF THESE ARE REQUIRED DO NOT TOUCH. 
#Takes a long time to download
import os
import sys
!DEBIAN_FRONTEND=noninteractive apt-get install -y tzdata
!pip install --upgrade "transformers>=4.49.0"
!apt-get update && apt-getinstall -y cuda
!!apt-get install libcurl4-openssl-dev
!apt-get update && apt-get install -y build-essential cmake
os.environ["PATH"] = "/usr/local/cuda-12.8/bin:" + os.environ.get("PATH", "")
os.environ["LD_LIBRARY_PATH"] = "/usr/local/cuda-12.8/lib64:" + os.environ.get("LD_LIBRARY_PATH", "")
!apt install -y nvidia-cuda-toolkit
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3
!pip install --no-deps unsloth
!pip install --pre --upgrade transformers
!apt-get install git-lfs
!git lfs install 
!git lfs track "*.gguf" 

In [ ]:
!pip install triton

In [ ]:
import os
os.environ["DISABLE_UNSLOTH_TRITON"] = "True"

In [ ]:
from unsloth import FastModel
import torch
import transformers

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",

    # Other popular models!
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-4b-it",
    max_seq_length = 2048, # Choose any for long context!
    attn_implementation="eager",
    load_in_4bit = False,  # 4 bit quantization to reduce memory
    load_in_8bit = True, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.35G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

Add LoRA adapters so we only need to update 1 to 10% of all parameters! This techniques allow us to fine-tune large models efficiently by adjusting only a subset of parameters instead of the entire model, reducing computational cost and memory usage.

In [ ]:

# Use PEFT (Parameter-Efficient Fine-Tunning) on our selected model
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 16,           # Larger = higher accuracy, but might overfit
    lora_alpha = 16,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth 2025.2.15 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


DATA PROCESSING Section

In [ ]:
import os
import json
import tempfile
import glob
from datasets import Dataset
from snowflake.snowpark.context import get_active_session

def convert_to_conversation(sample):
    conversation = [
        {
            "from": "system",
            "value": sample['instruction']
        },
        {
            "from": "human",
            "value": sample['input']
        },
        {
            "from": "gpt",
            "value": sample['output']
        }
    ]
    return {"conversations": conversation}

def load_data_from_snowflake_stage():
    # Create dataset list
    dataset = []
    
    # Get the active Snowpark session
    session = get_active_session()
    print(f"DEBUG: Got active Snowpark session")
    
    # Define schema and stage
    schema = "GIT"
    stage = "CARECONNECT_TRAINING_DATA_STAGE"
    fully_qualified_stage = f"{schema}.{stage}"
    print(f"DEBUG: Using fully qualified stage: {fully_qualified_stage}")
    
    # Set the schema for the session
    session.use_schema(schema)
    print(f"DEBUG: Set active schema to {schema}")
    
    # List files in the stage
    files_in_stage = session.sql(f"LIST @{fully_qualified_stage}").collect()
    print(f"DEBUG: Found {len(files_in_stage)} files in stage")
    for i, file_info in enumerate(files_in_stage[:5]):  # Print first 5 files
        print(f"DEBUG: File {i+1}: {file_info}")
    
    # Create a temporary directory to store downloaded files
    with tempfile.TemporaryDirectory() as temp_dir:
        print(f"DEBUG: Created temp directory: {temp_dir}")
        
        jsonl_files_count = 0
        processed_files_count = 0
        
        for file_info in files_in_stage:
            file_path = file_info["name"]
            file_name = os.path.basename(file_path)
            
            # Only process .jsonl files
            if not file_name.endswith(".jsonl"):
                continue
                
            jsonl_files_count += 1
            
            # Create a subdirectory for each file to avoid conflicts
            file_dir = os.path.join(temp_dir, f"file_{jsonl_files_count}")
            os.makedirs(file_dir, exist_ok=True)
            
            # Download file from stage to local temp directory
            print(f"DEBUG: Downloading {file_name} to {file_dir}")
            try:
                session.file.get(f"@{fully_qualified_stage}/{file_name}", file_dir)
                print(f"DEBUG: Download command executed for {file_name}")
            except Exception as e:
                print(f"ERROR: Failed to download {file_name}: {str(e)}")
                continue
            
            # Find the downloaded file(s)
            downloaded_files = glob.glob(os.path.join(file_dir, "**", "*"), recursive=True)
            print(f"DEBUG: Found {len(downloaded_files)} files in download directory")
            for df in downloaded_files[:5]:  # Print first 5 files
                print(f"DEBUG: Downloaded file: {df}")
            
            # Process each downloaded file
            jsonl_files = [f for f in downloaded_files if f.endswith(".jsonl") and os.path.isfile(f)]
            if not jsonl_files:
                print(f"WARNING: No .jsonl files found in download directory for {file_name}")
                continue
                
            for jsonl_file in jsonl_files:
                print(f"DEBUG: Processing {jsonl_file}")
                if os.path.exists(jsonl_file):
                    file_size = os.path.getsize(jsonl_file)
                    print(f"DEBUG: File size: {file_size} bytes")
                else:
                    print(f"ERROR: File doesn't exist: {jsonl_file}")
                    continue
                
                # Parse the downloaded JSONL file
                records_count = 0
                try:
                    with open(jsonl_file, "r", encoding="utf-8") as file:
                        for line_num, line in enumerate(file, 1):
                            try:
                                record = json.loads(line.strip())
                                dataset.append(record)
                                records_count += 1
                                
                                # Print sample of first record in each file
                                if line_num == 1:
                                    print(f"DEBUG: Sample record from {os.path.basename(jsonl_file)}:")
                                    # Print first few keys/values for sample
                                    sample_data = {k: str(v)[:50] + "..." if isinstance(v, str) and len(str(v)) > 50 else v 
                                                  for k, v in list(record.items())[:3]}
                                    print(f"DEBUG: {sample_data}")
                                    
                            except json.JSONDecodeError as e:
                                print(f"ERROR: Failed to parse JSON at line {line_num} in {jsonl_file}: {e}")
                except Exception as e:
                    print(f"ERROR: Failed to read {jsonl_file}: {str(e)}")
                    continue
                
                print(f"DEBUG: Processed {records_count} records from {os.path.basename(jsonl_file)}")
                processed_files_count += 1
        
        print(f"DEBUG: Processed {processed_files_count} out of {jsonl_files_count} JSONL files")
        print(f"DEBUG: Total records in dataset: {len(dataset)}")
    
    # Convert dataset to conversation format
    print("DEBUG: Converting to conversation format...")
    if not dataset:
        print("ERROR: No data was loaded into the dataset!")
        return Dataset.from_dict({"conversations": []})
        
    data_temp = [convert_to_conversation(data) for data in dataset]
    
    print(f"DEBUG: Created {len(data_temp)} conversation entries")
    
    # Print sample of conversation format
    if data_temp:
        print("DEBUG: Sample conversation format:")
        print(data_temp[0])
    
    data_dict = {
        "conversations": [item["conversations"] for item in data_temp]
    }
    
    print(f"DEBUG: Final dataset structure has {len(data_dict['conversations'])} conversations")
    
    # Create dataset from dictionary
    huggingface_dataset = Dataset.from_dict(data_dict)
    print(f"DEBUG: Created Hugging Face dataset with shape: {huggingface_dataset.shape}")
    
    return huggingface_dataset

# Load the dataset from Snowflake stage
print("Starting to load data from Snowflake stage...")
try:
    dataset = load_data_from_snowflake_stage()
    print(f"Dataset loaded successfully with {len(dataset)} entries")

    # Print dataset info
    print("\nDATASET INFO:")
    print(dataset)
    print("\nDATASET FEATURES:")
    print(dataset.features)

    # Show a sample from the dataset
    if len(dataset) > 0:
        print("\nSAMPLE FROM DATASET:")
        print(dataset[0])
except Exception as e:
    print(f"ERROR: Failed to load dataset: {str(e)}")
    import traceback
    traceback.print_exc()

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass


In [ ]:
print(dataset[:5])


{'conversations': [[{'from': 'system', 'value': 'Based on the following symptoms, predict the disease and recommend appropriate treatments.'}, {'from': 'human', 'value': 'Predict the disease based on these symptoms: Palpitations, Sweating, Trembling, Shortness of breath, Fear of losing control, Dizziness'}, {'from': 'gpt', 'value': 'Disease: Panic disorder; Treatments: Antidepressant medications, Cognitive Behavioral Therapy, Relaxation Techniques'}], [{'from': 'system', 'value': 'Based on the following symptoms, predict the disease and recommend appropriate treatments.'}, {'from': 'human', 'value': 'Predict the disease based on these symptoms: Hoarseness, Vocal Changes, Vocal Fatigue'}, {'from': 'gpt', 'value': 'Disease: Vocal cord polyp; Treatments: Voice Rest, Speech Therapy, Surgical Removal'}], [{'from': 'system', 'value': 'Based on the following symptoms, predict the disease and recommend appropriate treatments.'}, {'from': 'human', 'value': 'Predict the disease based on these sy

In [ ]:
from unsloth.chat_templates import standardize_data_formats
dataset = standardize_data_formats(dataset)

Standardizing format:   0%|          | 0/3085 [00:00<?, ? examples/s]

Map:   0%|          | 0/3085 [00:00<?, ? examples/s]

In [ ]:
def apply_chat_template(examples):
    texts = tokenizer.apply_chat_template(examples["conversations"])
    return { "text" : texts }
pass
dataset = dataset.map(apply_chat_template, batched=True, num_proc=1)

[{'content': 'Based on the following symptoms, predict the disease and recommend appropriate treatments.',
  'role': 'system'},
 {'content': 'Predict the disease based on these symptoms: Metabolic acidosis, apid breathing, rapid heart rate, confusion, headache, dizziness, seizures',
  'role': 'user'},
 {'content': 'Disease: Ethylene glycol poisoning-2; Treatments: Blood tests, Supportive Measures, Gastric Decontamination, Antidote Administration, Hemodialysis',
  'role': 'assistant'}]

In [ ]:
dataset[5]["text"] # verify that the formatting and preprocessing worked as expected.

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nBased on the following symptoms, predict the disease and recommend appropriate treatments.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nPredict the disease based on these symptoms: Metabolic acidosis, apid breathing, rapid heart rate, confusion, headache, dizziness, seizures<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nDisease: Ethylene glycol poisoning-2; Treatments: Blood tests, Supportive Measures, Gastric Decontamination, Antidote Administration, Hemodialysis<|eot_id|>'

Training setup


In [ ]:
from trl import SFTTrainer, SFTConfig
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    dataset_num_proc = 1,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        gradient_checkpointing=True,
        warmup_steps = 5,
        #num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 20,
        learning_rate = 2e-5, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_torch",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

Converting train dataset to ChatML (num_proc=2):   0%|          | 0/3085 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=2):   0%|          | 0/3085 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/3085 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=2):   0%|          | 0/3085 [00:00<?, ? examples/s]

In [ ]:
print(trainer.train_dataset[0])

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Map:   0%|          | 0/3085 [00:00<?, ? examples/s]

Very masking is done correctly

In [ ]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

'<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\nBased on the following symptoms, predict the disease and recommend appropriate treatments.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nPredict the disease based on these symptoms: Metabolic acidosis, apid breathing, rapid heart rate, confusion, headache, dizziness, seizures<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nDisease: Ethylene glycol poisoning-2; Treatments: Blood tests, Supportive Measures, Gastric Decontamination, Antidote Administration, Hemodialysis<|eot_id|>'

In [ ]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

'                                                                              \n\nDisease: Ethylene glycol poisoning-2; Treatments: Blood tests, Supportive Measures, Gastric Decontamination, Antidote Administration, Hemodialysis<|eot_id|>'

In [ ]:
#Show Computer memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
2.301 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 3,085 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 100
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss
1,5.895300
2,4.474600
3,3.438500
4,2.428900
5,2.038500
6,3.138600
7,1.976200
8,2.398200
9,1.797600
10,2.187700


More memory stats

In [ ]:
import torch

used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

364.4313 seconds used for training.
6.07 minutes used for training.
Peak reserved memory = 4.039 GB.
Peak reserved memory for training = 1.738 GB.
Peak reserved memory % of max memory = 27.4 %.
Peak reserved memory for training % of max memory = 11.79 %.


In [ ]:
#!git clone https://github.com/ggml-org/llama.cpp.git
#%cd llama.cpp
#!cmake -B build
#!cmake --build build --config Release

Local Saving

In [ ]:
model.save_pretrained("gemma-3")
tokenizer.save_pretrained("gemma-3")
model.config.save_pretrained("gemma-3")  # Ensure config is saved

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 2.4G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.27 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 28/28 [00:01<00:00, 23.01it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving careconnect-llama3.2-3b/pytorch_model-00001-of-00002.bin...
Unsloth: Saving careconnect-llama3.2-3b/pytorch_model-00002-of-00002.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


...

  0%|          | 0/1 [00:00<?, ?it/s]

unsloth.Q8_0.gguf:   0%|          | 0.00/3.42G [00:00<?, ?B/s]

Saved GGUF to https://huggingface.co/JdoubleU/careconnect-llama3.2-3b


No files have been modified since last commit. Skipping to prevent empty commit.


Saved Ollama Modelfile to https://huggingface.co/JdoubleU/careconnect-llama3.2-3b


In [ ]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained("gemma-3")
config.save_pretrained("gemma-3")

In [ ]:
!ls

In [ ]:
!ls gemma-3

In [ ]:
!pwd

In [ ]:
!ls /home/app/gemma-3

In [ ]:
from snowflake.snowpark.context import get_active_session
session = get_active_session()



put_result = session.file.put("/home/app/gemma-3/adapter_config.json","@SOFTWARESURGEONS_DB.GIT.CARECONNECT_GEMMA3_STAGE", auto_compress= False)
put_result = session.file.put("/home/app/gemma-3/tokenizer.json","@SOFTWARESURGEONS_DB.GIT.CARECONNECT_GEMMA3_STAGE", auto_compress= False)
put_result = session.file.put("/home/app/gemma-3/adapter_model.safetensors","@SOFTWARESURGEONS_DB.GIT.CARECONNECT_GEMMA3_STAGE", auto_compress= False)
put_result = session.file.put("/home/app/gemma-3/added_tokens.json","@SOFTWARESURGEONS_DB.GIT.CARECONNECT_GEMMA3_STAGE", auto_compress= False)
put_result = session.file.put("/home/app/gemma-3/chat_template.json","@SOFTWARESURGEONS_DB.GIT.CARECONNECT_GEMMA3_STAGE", auto_compress= False)
put_result = session.file.put("/home/app/gemma-3/config.json","@SOFTWARESURGEONS_DB.GIT.CARECONNECT_GEMMA3_STAGE", auto_compress= False)
put_result = session.file.put("/home/app/gemma-3/README.md","@SOFTWARESURGEONS_DB.GIT.CARECONNECT_GEMMA3_STAGE", auto_compress= False)
put_result = session.file.put("/home/app/gemma-3/preprocessor_config.json","@SOFTWARESURGEONS_DB.GIT.CARECONNECT_GEMMA3_STAGE", auto_compress= False)
put_result = session.file.put("/home/app/gemma-3/processor_config.json","@SOFTWARESURGEONS_DB.GIT.CARECONNECT_GEMMA3_STAGE", auto_compress= False)
put_result = session.file.put("/home/app/gemma-3/tokenizer.json","@SOFTWARESURGEONS_DB.GIT.CARECONNECT_GEMMA3_STAGE", auto_compress= False)
put_result = session.file.put("/home/app/gemma-3/tokenizer.model","@SOFTWARESURGEONS_DB.GIT.CARECONNECT_GEMMA3_STAGE", auto_compress= False)
put_result = session.file.put("/home/app/gemma-3/tokenizer_config.json","@SOFTWARESURGEONS_DB.GIT.CARECONNECT_GEMMA3_STAGE", auto_compress= False)

put_result[0].status

In [ ]:
ls @SOFTWARESURGEONS_DB.GIT.CARECONNECT_GEMMA3_STAGE
